<a href="https://colab.research.google.com/github/kochol/tf_tests/blob/master/test01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test 01

This is the first test of my tf code.

In [30]:
import tensorflow as tf
from tensorflow.keras import layers

# enable tf eager execution to support iterating
tf.enable_eager_execution()

print(tf.VERSION)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


## Create the model

Now create the model

In [0]:
inputs = tf.keras.Input(shape=(8,))  # Returns a placeholder tensor

# A layer instance is callable on a tensor, and returns a tensor.
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
predictions = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=predictions)

# I want to calc on float numbers so I use mse and mae for loss and metrics
# Also I have to find out which optimizer is good for me.
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='mse',
              metrics=['mae'])

## Load data from csv file

In [32]:
import pandas as pd
import numpy as np

# Download mydata.csv from github
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/kochol/tf_tests/master/mydata.csv

# load all data
mydata = pd.read_csv("mydata.csv")

print(mydata.columns)
print(mydata.dtypes)
print(mydata.shape)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  154k  100  154k    0     0   631k      0 --:--:-- --:--:-- --:--:--  631k
Index(['start', 'open', 'high', 'low', 'close', 'vwp', 'volume', 'trades'], dtype='object')
start       int64
open      float64
high      float64
low       float64
close     float64
vwp       float64
volume    float64
trades      int64
dtype: object
(1884, 8)


Prepare the data for training

In [33]:
train = mydata.reindex(index=mydata[0:-1])
print(train.shape)

test = pd.DataFrame({'price': mydata[1:].close})
print(test.columns)
print(test.dtypes)
print(test.shape)

(1883, 8)
Index(['price'], dtype='object')
price    float64
dtype: object
(1883, 1)


## Train the model

In [36]:
model.fit(train.to_numpy(), test.to_numpy(), epochs=10, batch_size=32)

Epoch 1/10
1883/1883 [==============================] - 0s 94us/sample - loss: nan - mean_absolute_error: nan
Epoch 2/10
1883/1883 [==============================] - 0s 45us/sample - loss: nan - mean_absolute_error: nan
Epoch 3/10
1883/1883 [==============================] - 0s 45us/sample - loss: nan - mean_absolute_error: nan
Epoch 4/10
1883/1883 [==============================] - 0s 45us/sample - loss: nan - mean_absolute_error: nan
Epoch 5/10
1883/1883 [==============================] - 0s 42us/sample - loss: nan - mean_absolute_error: nan
Epoch 6/10
1883/1883 [==============================] - 0s 43us/sample - loss: nan - mean_absolute_error: nan
Epoch 7/10
1883/1883 [==============================] - 0s 44us/sample - loss: nan - mean_absolute_error: nan
Epoch 8/10
1883/1883 [==============================] - 0s 40us/sample - loss: nan - mean_absolute_error: nan
Epoch 9/10
1883/1883 [==============================] - 0s 42us/sample - loss: nan - mean_absolute_error: nan
Epoch 10/1